In [2]:
import os
import subprocess
import time
import sys
sys.path.append('/workspace/activation-steering')

In [3]:
# Start Xvfb manually (xvfb-run does this under the hood)
subprocess.Popen([
    'Xvfb', ':99', '-screen', '0', '1280x1024x24', '+extension', 'GLX', '+render', '-noreset'
])

time.sleep(2)  # Give Xvfb time to start

# Set environment variables to match your working terminal setup
os.environ['DISPLAY'] = ':99'
os.environ['LIBGL_ALWAYS_SOFTWARE'] = '1'
os.environ['LIBGL_DRIVERS_PATH'] = '/usr/lib/x86_64-linux-gnu/dri'
os.environ['LD_PRELOAD'] = '/usr/lib/x86_64-linux-gnu/libffi.so.7'

In [4]:
!glxinfo | grep 'OpenGL renderer string'
!glxinfo | grep 'OpenGL version string'

ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
OpenGL renderer string: llvmpipe (LLVM 15.0.7, 256 bits)
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
OpenGL version string: 4.5 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3


In [5]:
!glxinfo | grep 'GLX version'
!glxinfo | grep 'renderer'

ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
GLX version: 1.4
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libffi.so.7' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
    GLX_MESA_copy_sub_buffer, GLX_MESA_query_renderer, GLX_MESA_swap_control, 
    GLX_MESA_query_renderer, GLX_OML_swap_method, GLX_SGIS_multi

In [ ]:
import openai
from arguments import get_config
from interfaces import setup_LMP
from visualizers import ValueMapVisualizer
from envs.rlbench_env import VoxPoserRLBench
from utils import set_lmp_objects
import numpy as np
from rlbench import tasks

  # set your API key here

In [5]:
config = get_config('rlbench')
# uncomment this if you'd like to change the language model (e.g., for faster speed or lower cost)
# for lmp_name, cfg in config['lmp_config']['lmps'].items():
#     cfg['model'] = 'gpt-3.5-turbo'

# initialize env and voxposer ui
visualizer = ValueMapVisualizer(config['visualizer'])
env = VoxPoserRLBench(visualizer=visualizer)
print("done visualizer")

##################################################
## voxel resolution: [0.0105 0.0131 0.01  ]
##################################################




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

: 

rlbench.tasks.set_the_table.SetTheTable

In [19]:
task = env.rlbench_env.get_task(tasks.SetTheTable)
task

In [20]:
task.get_name()

'set_the_table'

In [ ]:
lmps, lmp_env = setup_LMP(env, config, debug=False)
voxposer_ui = lmps['plan_ui']

## Playground

By default we use one of the instructions that come with each task. However, you may treat each task as simply a scene initialization from RLBench, and feel free to try any task you can come up with for the scene.

Note:
- Whether an instruction can be executed or not depends on 1) whether relevant objects are available, and 2) capabilities of the overall algorithm.
- Each execution may produce one or more visualizations. You may view them in "./visualizations/" folder.
- The prompts are adapted with minimal change from the real-world environment in the VoxPoser paper. If a task failure is due to incorrect code generated by the LLM, feel free to modify the relevant prompt in "./prompts/" folder.
- You may view the reward by printing "env.latest_reward". These are computed by RLBench for each task.
- To inspect in viewer without performing any action, you may call "env.rlbench_env._scene.step()" in a loop.

In [6]:
# # uncomment this to show all available tasks in rlbench
# # NOTE: in order to run a new task, you need to add the list of objects (and their corresponding env names) to the "task_object_names.json" file. See README for more details.
# print([task for task in dir(tasks) if task[0].isupper() and not '_' in task])

In [7]:
# below are the tasks that have object names added to the "task_object_names.json" file
# uncomment one to use
env.load_task(tasks.PutRubbishInBin)
# env.load_task(tasks.LampOff)
# env.load_task(tasks.OpenWineBottle)
# env.load_task(tasks.PushButton)
# env.load_task(tasks.TakeOffWeighingScales)
# env.load_task(tasks.MeatOffGrill)
# env.load_task(tasks.SlideBlockToTarget)
# env.load_task(tasks.TakeLidOffSaucepan)
# env.load_task(tasks.TakeUmbrellaOutOfUmbrellaStand)

descriptions, obs = env.reset()
set_lmp_objects(lmps, env.get_object_names())  # set the object names to be used by voxposer

In [8]:
instruction = np.random.choice(descriptions)
voxposer_ui(instruction)

KeyError: 'use_model'

In [ ]:
env.latest_reward

1.0